# Chapter 8: Dimensionality Reduction

Bab ini membahas masalah yang dikenal sebagai **Curse of Dimensionality** (Kutukan Dimensi), di mana banyak algoritma Machine Learning berkinerja buruk pada dataset dengan jumlah fitur yang sangat tinggi (dimensi tinggi).

Kita akan mengeksplorasi teknik-teknik untuk mengurangi jumlah fitur (dimensi) ini, yang dapat mempercepat pelatihan secara signifikan dan terkadang bahkan meningkatkan kinerja model. Mengurangi dimensi juga sangat berguna untuk visualisasi data.

## The Curse of Dimensionality

Masalah utama dalam ruang berdimensi tinggi adalah data menjadi sangat tersebar (*sparse*). Sebagian besar *instance* pelatihan kemungkinan besar akan berjauhan satu sama lain, membuat prediksi menjadi kurang andal karena didasarkan pada ekstrapolasi yang jauh lebih besar. Ini meningkatkan risiko *overfitting*.

## Pendekatan Utama untuk Pengurangan Dimensi

Ada dua pendekatan utama untuk mengurangi dimensi:

1.  **Projection (Proyeksi):** Sebagian besar data di dunia nyata tidak tersebar secara seragam di semua dimensi. Seringkali, data berada di dekat *subspace* berdimensi jauh lebih rendah. Proyeksi bekerja dengan memproyeksikan setiap *instance* data ke *subspace* ini. Teknik yang paling populer untuk ini adalah **Principal Component Analysis (PCA)**.

2.  **Manifold Learning:** Pendekatan ini mengasumsikan bahwa sebagian besar dataset berdimensi tinggi di dunia nyata berada di dekat *manifold* berdimensi jauh lebih rendah. *Manifold* adalah bentuk yang dapat ditekuk dan diputar dalam ruang berdimensi lebih tinggi (contoh: Swiss roll). Manifold Learning mencoba untuk "membuka gulungan" *manifold* ini. Contoh tekniknya adalah **Locally Linear Embedding (LLE)**.

## PCA (Principal Component Analysis)

PCA adalah algoritma pengurangan dimensi yang paling populer. Ia bekerja dengan mengidentifikasi *hyperplane* yang paling dekat dengan data, lalu memproyeksikan data ke *hyperplane* tersebut.

PCA memilih sumbu (disebut *principal component*) yang mempertahankan jumlah varians maksimum dalam data. Sumbu kedua dipilih ortogonal terhadap yang pertama dan mempertahankan sisa varians terbesar, dan seterusnya.

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

# Membuat dataset 3D
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1
angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

# Menggunakan PCA untuk mengurangi dimensi menjadi 2
pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

print("Principal components:")
print(pca.components_)

print("Explained variance ratio:")
print(pca.explained_variance_ratio_)

### Kernel PCA (kPCA)

Seperti yang kita lihat di bab SVM, *kernel trick* adalah teknik matematis yang memungkinkan kita memetakan *instance* ke ruang berdimensi sangat tinggi secara implisit. Trik yang sama dapat diterapkan pada PCA, yang memungkinkan proyeksi non-linear untuk pengurangan dimensi.

Ini disebut **Kernel PCA (kPCA)**. Ini sangat baik dalam mempertahankan klaster *instance* setelah proyeksi atau bahkan membuka dataset yang terletak di dekat *manifold* yang terpilin.

In [ ]:
from sklearn.decomposition import KernelPCA
from sklearn.datasets import make_swiss_roll

X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)

# Menggunakan kPCA dengan kernel RBF
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

## LLE (Locally Linear Embedding)

**LLE** adalah teknik *Manifold Learning* yang tidak bergantung pada proyeksi. LLE bekerja dengan terlebih dahulu mengukur bagaimana setiap *instance* pelatihan berhubungan secara linear dengan tetangga terdekatnya. Kemudian, ia mencari representasi berdimensi rendah dari set pelatihan di mana hubungan lokal ini paling terjaga.

Pendekatan ini membuatnya sangat baik dalam membuka *manifold* yang terpilin, terutama jika tidak ada terlalu banyak *noise*.

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced_lle = lle.fit_transform(X)